In [44]:
import libpypack
from libpypack.locations import map_locations
from libpypack.visualization import generate_maps
from libpypack.visualization import heatmap
from libpypack.visualization import choropleth

import gmplot
import docker
import requests
import tarfile
import os
import geopandas

In [59]:
import pandas as pd
# df = pd.read_csv("/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/test_data/Swing_States.csv")
# df
names = ["TweetID", "Timestamp", "Full_Text", "In_Reply_To_User_ID", "User_ID", "User_Name", "User_Screen_Name",
             "Coordinates", "Place", "Bounding_Box", "Quoted_Status_ID", "Retweeted_Status", "Hashtags", "URLs", "User_Mentions",
             "Language", "Media"]
df=pd.read_csv("/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/test_data/aggregate_political_file_5000.csv", sep='\t')
del df["Unnamed: 0.1"]
del df["Language"]
df


,Unnamed: 0,TweetID,Timestamp,Full_Text,In_Reply_To_User_ID,User_ID,User_Name,User_Screen_Name,Coordinates,Place,Bounding_Box,Quoted_Status_ID,Retweeted_Status,Hashtags,URLs,User_Mentions,Media
0,1158127,2018-11-01 18:23:59,b'Alime Hay\xc4\xb1rl\xc4\xb1 Cumalar demi\xc5...,NaN,3065861716,@gülayşeçınar,GulayseCinar,NaN,NaN,[],0,1058022635697815556,[],[],{'SeyhEdebali1': 903432832386813955},[],tr
1,19458875,2018-11-09 07:52:44,b'#Kurallar\xc4\xb1mVarBenim sen se\xc3\xa7 be...,NaN,1047345066459758592,Denizata,Denizat70197410,NaN,NaN,[],0,[],['KurallarımVarBenim'],[],{},[],tr
2,949369,2018-11-01 16:31:25,b'\xf0\x9f\x8c\x8f TOP COUNTRIES TWEETING \xe2...,NaN,884648405754028032,💜SPEAK YOURSELF💜,Rose241293,NaN,NaN,[],0,1058023718893498368,"['MAMAVOTE', 'BTS']",[],{'btsvotingteam': 921338490775773184},[],en
3,3516929,2018-11-02 17:53:41,b'@wqsley @notMigu @luanpereiradev @CdNLeon @S...,3.987787e+09,901264397829537793,Brasileiro,PauloBCarneiro,NaN,NaN,[],0,[],[],[],"{'CdNLeon': 37312877, 'wqsley': 3987787245, 'f...",[],pt
4,30497526,2018-11-14 08:55:51,b'BREAKING: Democrat Josh Harder wins U.S. Hou...,NaN,19686263,Dr HerbSistah,Herbsistah,NaN,NaN,[],0,1062533611751202817,[],[],{'AP': 51241574},[],en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5661701,2018-11-03 15:42:59,b'@shivbhaktajoshi @PeacefulOneVA @Craigersct5...,NaN,3243479167,Ashish,stoplootofusa,NaN,NaN,[],0,1058615089731780608,[],[],"{'arthur_oslund': 3968865322, 'FirmLogix': 185...",[],en
4996,28702632,2018-11-13 13:23:40,b'Fact: Out of the 230 BJP candidates contesti...,NaN,911512843018657793,Afzal Congress Cordinator I.T Cell Loni,AfzalNaqvi3,NaN,NaN,[],0,1062183134907846666,[],[],{'srivatsayb': 45261768},[],en
4997,19899388,2018-11-09 12:34:39,b'Towering firenado seen swirling amid the #Ca...,NaN,889183529321549824,D’az🥀🏴,o5views,NaN,NaN,[],0,1060872480590626821,['CampFire'],[],{'ABC': 28785486},[],en
4998,24757117,2018-11-11 18:48:57,b'CW: Politics 3/5 Yet because of that libs ar...,3.127985e+09,3127985231,Gentle Ice 💜,GameswoGravity,NaN,NaN,[],0,[],[],['https://twitter.com/i/web/status/10616922732...,{},[],en


In [61]:
df.columns = names
df[0:2000].to_csv('political_2000_sample.csv', sep='\t')

In [19]:
tweet_df = map_locations.locations_df("/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/test_data/Swing_States.csv", output_dir="/Users/thatcher/Desktop", df_column="Text", sep=' ')
# over_gdf = generate_maps.generate_overlay_gdf(tweet_df, filename="/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/libpypack/examples/states_21basic/states.shp")
# generate_maps.plot_gdf(over_gdf)
# heatmap.heatmap(loc_gdf)
# choropleth.choropleth_map(loc_gdf, shp_path="/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/libpypack/examples/states_21basic/states.shp")

In [22]:
loc_gdf = generate_maps.get_loc_gdf(tweet_df)

In [33]:
import folium
import os
from folium.plugins import HeatMap


def heatmap(loc_gdf, column="Location Extracted", heat_value=None, normalize_data=False, output_name='heatmap.html', output_dir=''):
    locations = loc_gdf[column].value_counts().values
    max_amount = float(locations.max())

    if(normalize_data):
        locations = loc_gdf['Location Extracted'].value_counts().values
        print(locations)
        normalized = (locations-locations.min())/(locations.max()-locations.min())

        hmap = folium.Map(zoom_start=7)

        hm_wides = HeatMap( list(zip(loc_gdf.Latitude.values, loc_gdf.Longitude.values, normalized)),
                           min_opacity=0.2,
                           radius=17, blur=15,
                           max_zoom=1,
                         )

        hmap.add_child(hm_wides)
        hmap.save(os.path.join(output_dir, output_name))

        return hmap

    else:

        max_amount = float(locations.max())

        hmap = folium.Map(zoom_start=7)
        print(list(zip(loc_gdf.Latitude.values, loc_gdf.Longitude.values, locations)))
        hm_wides = HeatMap( list(zip(loc_gdf.Latitude.values, loc_gdf.Longitude.values)),
                           min_opacity=0.2,
                           max_val=max_amount,
                           radius=17, blur=15,
                           max_zoom=1,
                         )

        hmap.add_child(hm_wides)
        hmap.save(os.path.join(output_dir, output_name))

        return hmap


heat = heatmap(loc_gdf)
heat

[(42.00027, -93.50049, 1), (44.25029, -85.50033, 1), (46.25024, -94.25055, 1), (40.25034, -83.00018, 1), (39.25021, -116.75119, 1), (43.66702, -71.4998, 1), (35.50069, -80.00032, 1), (40.27245, -76.90567, 1), (44.50024, -90.00041, 1)]


In [ ]:
choropleth.choropleth_map(loc_gdf, shp_path="/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/libpypack/examples/states_21basic/states.shp")

In [10]:
heat

NameError: name 'heattttmap' is not defined